## Microtubule Dynamics ABM Julia notebook
This notebook is intended as a development and testing file , so i will change regulary.

nächstes meeting 05.10 10:45

### _TODO_ ::

    - Drei Abbildungen erstellen für klein 100, mittel300, groß1000 ,a 50 mal begrenzte randbedingungen,
    - seed funktion?!
    - Standartdabweichung (korridor) 
    - Rohdaten gut aufbewahren, sinnvolle beschriftungen  
    - Periodischen Randbedingugen mit in microtubuli modell implementieren

  - Wachstumsgeswindigkeit (erstmal ohne Depolymerisation) ~ N.Agenten/Gridsize & P-polymerisiert


In [ ]:
using Agents
using CairoMakie
using Distributions
#import DrWatson: @dict

In [ ]:
#= The agent class representing one tubulin dimer
    those can polymerize and also hydrolize from GTP to GDP
=#

@agent tubulin GridAgent{2} begin
    polym::Bool
    GDP::Bool
end

In [ ]:
#= Main Function to initialize the model=#

function initialize(;Nstarts=1, numagents = 550, 
        griddims = (100,100), 
        p_hyd= 0.001, 
        p_polym = 0.99,
        p_depolym_GTP= 0.005,
        p_depolym_GDP=0.01 )
   
    # Probability to hydrolize from GTP to GDP: 
    P_hyd = Binomial(1,p_hyd)
    # Probability to Polymerize/bind to the microtuble 
    P_polym = Binomial(1,p_polym)
    # Probability to depolimerize if not hydrolized (GTP - tubulin) = lower
    P_depolym_GTP = Binomial(1,1-p_depolym_GTP)
    # Probability to depolimerize if hydrolized (GDP - tubulin) = higher
    P_depolym_GDP = Binomial(1,1-p_depolym_GDP)
    
    
    properties = Dict(:griddims=> griddims,
                    :numagents => numagents,
                    :Nstarts => Nstarts,
                    :p_hyd => p_hyd,
                    :p_polym => p_polym,
                    :p_depolym_GTP => p_depolym_GTP,
                    :p_depolym_GDP => p_depolym_GDP,
                    :P_hyd => P_hyd,
                    :P_polym=> P_polym,
                    :P_depolym_GTP=>  P_depolym_GTP,
                    :P_depolym_GDP=> P_depolym_GDP
                    )
    
    
    space = GridSpace(griddims, periodic= false, metric = :chebyshev)
    
    model = ABM(
        tubulin,space;
        scheduler = Schedulers.randomly,
        properties = properties
    )
    # populize Space
    id = 0
    # Startingpoints:
    for _ in 1:Nstarts
        id += 1
        agent =  tubulin(id,(50,50),true,true)
        if id == 1
            add_agent_pos!(agent,model) # add first staringpoint to center
        else
            add_agent_single!(agent,model) # add additional startingpoints to random positions 
        end
    end
    # unpolimerized (free) agents
    for _ in 1:numagents 
        id += 1
        agent =  tubulin(id,(1,1),false,false)
        add_agent_single!(agent,model) # add agents to random positions
    end
    return model
end

In [ ]:
# peroidic agent step function
function agent_step!(agent,model)
    # bounce check für agenten (1|100 , y) | (x, 1|100) in eigene funktion aussiedeln
   if agent.id > model.Nstarts  
        # Logic for unpolymerized agents:
        if agent.polym == false      
            if agent.pos[1]>1 && agent.pos[2]>1
                if isempty(tuple(agent.pos[1]-1,agent.pos[2]-1),model)
                    walk!(agent, rand, model, ifempty =true)
                end
            elseif agent.pos[1]==1 && agent.pos[2]>1
                if isempty(tuple(model.griddims[1],agent.pos[2]-1),model)
                    walk!(agent, rand, model, ifempty =true)
                end    
            elseif agent.pos[1]>1 && agent.pos[2]==1 
                if isempty(tuple(agent.pos[1]-1,model.griddims[2]),model)
                    walk!(agent, rand, model, ifempty =true)
                end    
            elseif agent.pos[1]==1 && agent.pos[2]==1 
                if isempty(tuple(model.griddims[2],model.griddims[2]),model)
                    walk!(agent, rand, model, ifempty =true)
                end 
            elseif agent.pos[1] == model.griddims[1] && agent.pos[2]>1
                if isempty(tuple(1, agent.pos[2]-1))
                     walk!(agent, rand, model, ifempty =true)
                end 
            elseif agent.pos[1]>1 && agent.pos[2] == model.griddims[2]
                if isempty(tuple(agent.pos[2]-1,1 ))
                     walk!(agent, rand, model, ifempty =true)
                end 
            end
        for id in nearby_ids(agent, model,1) # 
            if  model[id].property == false
                # normal case
               if model[id].pos[1] == agent.pos[1] + 1 && model[id].pos[2] == agent.pos[2] + 1 
                  agent.property = false
                  # x 1->100  
                elseif model[id].pos[1] == 1 && agent.pos[1] == 100 && model[id].pos[2] == agent.pos[2]+1 
                  agent.property = false
                # y 1->100
                elseif model[id].pos[1] == agent.pos[1] + 1 && model[id].pos[2] == 1 && agent.pos[2] == 100 
                  agent.property = false
                # x & y 1 ->100
                elseif model[id].pos[1] == 1 && agent.pos[1] == 100 && model[id].pos[2] == 1 && agent.pos[2] == 100
                    agent.property = false
                end
            
           end
       end
    end
end

In [ ]:
# agent step func
function agent_step!(agent,model)
    # Starting points have to stay the same
    if agent.id > model.Nstarts  
        # Logic for unpolymerized agents:
        if agent.polym == false            
            walk!(agent, rand, model, ifempty =true)# brownsche bewegung wenn nicht polymerisiert
            for id in nearby_ids(agent, model, [(1, 1:1), (2, 1:1)]) # check topright neighbour
                # Problem wenn periodisch metrik tuple arg Only possible with Chebyshev spaces.
              #for id in nearby_ids(agent, model,1)
                if model[id].polym # if polymerized
                    agent.polym = rand(model.P_polym,1)[1] # polimerize with P_polym
                end
            end
        end
        # Logic for polymierized agents:
        if agent.polym
            if agent.GDP == false   # if not hydrolized
                agent.GDP = rand(model.P_hyd,1)[1] # hydrolize with P_hyd
            end
            if isempty(tuple(agent.pos[1]-1,agent.pos[2]-1),model) # check if end of microtuble
             # Problem wenn periodisch metrik "attempt to access 100×100 Matrix{Vector{Int64}} at index [13, 0]""

                if agent.GDP  # if hydrolized
                        agent.polym = rand(model.P_depolym_GDP,1)[1] # depolyimerize with P_depolym_GDP
                    else
                        agent.polym = rand(model.P_depolym_GTP,1)[1] # depolymerize with P_depolym_GDP     
                end
            end
        end
    end 
end

In [ ]:
# model-Step func:
function model_step!(model)
    # TODO: calculate mean squared displacement MSD for every tick
    # calculate mean microtuble length 

end

In [ ]:
# initialization
model = initialize(
    Nstarts=3,
    numagents = 300,
    p_polym=0.99,
    p_hyd= 0.015,
    p_depolym_GTP=0.01,
    p_depolym_GDP=0.06)


In [ ]:
#Video out:
using InteractiveDynamics
using GLMakie
# coloring

groupcolor(a) = a.polym == true ? :red : :blue
groupmarker(a) = a.GDP == true ? :circle : '○'

#GLMakie.activate!()

abm_video(
    "Microtubule_continuus.mp4", model, agent_step!,model_step!;
    ac = groupcolor, am = groupmarker,
    framerate = 10, frames = 1200,
    title = "Microtubule Simulation:"
)


In [ ]:
### Data collection

adata = [:pos, :polym, :GDP]

model = initialize(griddims =(50,50),
    Nstarts=0,
    numagents = 40,
    p_polym=0.99,
    p_hyd= 0.0,
    p_depolym_GTP=0.0,
    p_depolym_GDP=0.0)

data, _ = run!(model,agent_step!, model_step!,100;adata)




In [ ]:
# MSD function
function MSD(data)   
    n_ids= maximum(data.id)
    n_steps=maximum(data.step)+1
    r=1:n_ids:n_steps*n_ids+n_ids
    poses=hcat(collect.(data.pos)...)
    x_0=poses[1,unique(data.id)]
    y_0=poses[2,unique(data.id)]
    MSD = zeros(n_steps) #array länge data.step init zeros()
        for i in 1:n_steps
            MSD[i]=mean(sqrt.((poses[1,r[i]:r[i+1]-1].-x_0).^2 + (poses[2,r[i]:r[i+1]-1].-y_0).^2))
        end
    return MSD
end

In [ ]:
MSD(data)

In [ ]:
## Ensemble run
models = [initialize(; Nstarts=0,
        numagents=200,
        griddims=(x,x), 
        p_hyd=0,
        p_polym=0,
        p_depolym_GTP=0,
        p_depolym_GDP=0) for x in [100,300,1000]];

adata = [:pos, :polym, :GDP]

adf, = ensemblerun!(models, agent_step!, model_step!, 500; adata)

In [ ]:
## MSD for multiple ensembles

MSDS = DataFrame(Step= levels(adf.step))
for i in levels(adf.ensemble)
    name = Symbol("ens_$i")
    MSDS[!,name] = MSD(adf[(adf.ensemble .== i ),:])
end
MSDS

In [ ]:
#delete first row of df


In [ ]:
# MSD plots
MSDS = MSDS[setdiff(1:end, 1), :]#delete 0 timestep before plotting since log(0) not defined.
f = Figure()
Axis(f[1, 1],yscale=log10,xscale=log10)
#Axis(f[1,1])
lines!(MSDS.ens_1)
lines!(MSDS.ens_2)
lines!(MSDS.ens_3)
lines!(MSDS.ens_4)
lines!(MSDS.ens_5)
lines!(MSDS.ens_6)
lines!(MSDS.ens_7)
lines!(MSDS.ens_8)
lines!(MSDS.ens_9)
lines!(MSDS.ens_10)
f


In [ ]:
# Interactive exploration:
using Statistics
using CairoMakie
using InteractiveDynamics
using GLMakie
# parameter Range for sliders
model = initialize(; numagents = 600,Nstarts=4)

parange = Dict(
    :Nstarts => 1:30,
    :numagents => 100:1000,
    :p_hyd =>0.0:0.01:1.0, 
    :p_polym => 0.0:0.01:1.0, 
    :p_depolym_GTP =>0.0:0.01:1.0,
    :p_depolym_GDP => 0.0:0.01:1.0,)


groupcolor(a) = a.polym == true ? :red : :blue
groupmarker(a) = a.GDP == true ? :circle : '○'


adata = [(:polym,sum), (:GDP,sum)]
alabels = ["Polimerized", "Hydrolized"]



figure, adf, mdf = abm_data_exploration( 
    model, agent_step!, model_step!, parange;
    ac = groupcolor, am = groupmarker, as = 10,
    adata, alabels
)

